In [1]:
from econ_regime.econ_regime import *
from price.macro_info import get_econ_info
from price.price_processing import *
from strategy.factors.momentum import *
from strategy.factors.volatility import *
from strategy.factors.beta import *
from strategy.optimize.cross_sectional import *
from backtest.metric import *

import pandas as pd
import numpy as np
import statsmodels.api as sm
import bs4 as bs
import requests
import datetime as dt
import pickle5 as pickle
import seaborn as sns

In [2]:
sp500_daily_ret = pd.read_csv('result/sp500_daily_rets.csv', index_col=0)
beta_daily_rets = pd.read_csv('result/beta_daily_rets.csv', index_col=0)
mom_daily_rets = pd.read_csv('result/mom_daily_rets.csv', index_col=0)
vol_daily_rets = pd.read_csv('result/vol_daily_rets.csv', index_col=0)

In [3]:
factor_daily_df = pd.concat([sp500_daily_ret, beta_daily_rets, mom_daily_rets, vol_daily_rets], axis=1)
factor_daily_df.columns = ['sp500', 'beta', 'mom', 'vol']
factor_daily_df.index = pd.to_datetime(factor_daily_df.index)
factor_daily_df.index.name = 'date_time'
factor_daily_df.fillna(0, inplace=True)

factor_monthly_rets = factor_daily_df.resample('M').last()

In [4]:
invest_price = (1 + factor_daily_df).cumprod().ffill()
invest_price = add_cash(invest_price, 252, 0.04)
invest_price

rebal_list = rebal_dates(invest_price, period='month')
rebal_price_df = price_on_rebal(invest_price, rebal_list)

In [5]:
monthly_index = rebal_dates(factor_daily_df, period='month')

In [6]:
def rolling_sharpe(rets_df, monthly_index):
    sharp_df = pd.DataFrame()
    for index in monthly_index:
        df = rets_df.loc[:index, :]
        df = df.iloc[-252:, :] if len(df) >= 252 else df
        
        weights = np.array([i for i in range(1, len(df)+1)])
        weights = weights / weights.mean()

        df = df.apply(lambda x: x * weights, axis=0)

        temp = df.rolling(len(df)).mean() / df.rolling(len(df)).std()
        #print(type(temp.dropna(axis=0)))
        sharp_df = pd.concat([sharp_df,temp.dropna(axis=0)], axis=0)
        
    return sharp_df

def max_to_1(row):
    max_value = row.max()
    row[row == max_value] = 1
    row[row != 1] = 0
    return row

In [7]:
rolling_sharp_df = rolling_sharpe(factor_daily_df, rebal_list)

In [8]:
signal = rolling_sharp_df.apply(max_to_1, axis=1)
signal.shift(1).fillna(0, inplace=True)

In [9]:
regime_weight = Equalizer(signal=signal, rebal_price=factor_monthly_rets, param=12).ew()
regime_weight = regime_weight.fillna(0)
regime_weight.isna().all()

TypeError: __init__() got an unexpected keyword argument 'rebal_price'

In [10]:
regime_port = calculate_portvals(price_df=invest_price, weight_df=regime_weight, signal_df=signal, long_only=True)

NameError: name 'regime_weight' is not defined

In [11]:
port_rets(regime_port, True).plot(lw=1)
Metric(portfolio=port_rets(regime_port, True)).print_report()

NameError: name 'regime_port' is not defined